# **CAFA 6 Protein Fonksiyon Tahmini - Öğretici ve Zenginleştirilmiş Keşifsel Veri Analizi (EDA)**

### **Team:**
**DTU Proteomics Core**

### **Author:**
**Olaf Yunus Laitinen Imanov**  
*Data Scientist in Proteomics*  
DTU Proteomics Core  
DTU Bioengineering, Department of Biotechnology and Biomedicine  
Technical University of Denmark (Danmarks Tekniske Universitet)  
Søltofts Plads, Building 224, Room 017, 2800 Kgs. Lyngby  
olyulaim@dtu.dk  
+46 76 236 80 88  

---

### **Giriş: Proteinlerin Gizemli Dünyasına Yolculuk**

Bu notebook, **CAFA 6 Protein Fonksiyon Tahmini** yarışmasının veri setini analiz etmekle kalmayıp, aynı zamanda biyoinformatik ve proteomik dünyasına bir giriş yapmayı amaçlamaktadır. Amacımız, bir proteinin amino asit diziliminden yola çıkarak onun biyolojik görevini nasıl tahmin edebileceğimizi anlamak ve bu süreçte kullanılan temel kavramları ve veri bilimi tekniklerini açıklamaktır.

**Neden Protein Fonksiyonu Tahmini?**

Genom projeleri sayesinde elimizde milyonlarca proteinin amino asit dizilimi bilgisi var. Ancak bu proteinlerin büyük bir kısmının ne iş yaptığı, yani "fonksiyonu" hala bilinmiyor. Bu fonksiyonları laboratuvarda deneysel olarak belirlemek zaman alıcı ve maliyetlidir. İşte bu noktada **biyoinformatik** ve **makine öğrenmesi** devreye giriyor. Hesaplamalı yöntemler kullanarak, bir proteinin diziliminden onun fonksiyonunu tahmin etmek, biyolojik araştırmaları hızlandırabilir, hastalıkların mekanizmalarını aydınlatabilir ve yeni ilaç hedeflerinin keşfedilmesine olanak tanıyabilir. Bu yarışma, tam da bu önemli bilimsel probleme odaklanmaktadır.

---

### **Analiz Planı:**

1.  **Bölüm 1: Hazırlık ve Kavramsal Temeller**
    *   Gerekli Python kütüphanelerinin yüklenmesi.
    *   **Kavram Köşesi:** Protein, Amino Asit, Gene Ontology (GO) ve Taksonomi nedir?
2.  **Bölüm 2: Veri Setini Tanıyalım**
    *   Yarışma dosyalarına genel bir bakış ve görevimizin tanımı.
3.  **Bölüm 3: Protein Sekansları Analizi (`train_sequences.fasta`)**
    *   **Bilgi:** Proteinler neden farklı uzunluklardadır? Amino asit dağılımı bize ne anlatır?
    *   Sekans uzunlukları, amino asit frekansları ve bunların biyolojik anlamları.
4.  **Bölüm 4: Fonksiyonel Etiketler Analizi (`train_terms.tsv`)**
    *   **Bilgi:** "Çok etiketli sınıflandırma" nedir? "Etiket dengesizliği" modelimizi nasıl etkiler?
    *   GO terimlerinin dağılımı ve istatistiksel analizi.
5.  **Bölüm 5: Taksonomi Analizi (`train_taxonomy.tsv`)**
    *   **Bilgi:** Model organizmalar neden önemlidir? Evrimsel bilgi fonksiyon tahmininde nasıl kullanılır?
    *   Proteinlerin ait olduğu canlı türlerinin incelenmesi.
6.  **Bölüm 6: Gene Ontology (GO) Hiyerarşisi (`go-basic.obo`)**
    *   **Bilgi:** Yönlendirilmiş Döngüsüz Graf (DAG) nedir? Fonksiyonlar arasındaki "is_a" ilişkisi ne anlama gelir?
    *   GO grafının yapısı, terim derinliği ve Bilgi Birikimi (IA) arasındaki ilişki.
7.  **Bölüm 7: Bütünsel Bakış ve Karşılaştırmalar**
    *   Eğitim ve test setleri arasındaki benzerliklerin önemi: "Dağılım Kayması (Domain Shift)" kavramı.
8.  **Bölüm 8: İleri Düzey Görselleştirmeler**
    *   En popüler GO terimleri arasındaki hiyerarşik ilişkinin interaktif bir ağ grafiği ile görselleştirilmesi.
9.  **Bölüm 9: Sonuç ve Modelleme Stratejileri**
    *   Analizden elde edilen bulguların özetlenmesi ve başarılı bir model geliştirmek için yol haritası.

---

### **1. Hazırlık: Kütüphanelerin Yüklenmesi ve Kavramsal Temeller**

#### **1.1 Gerekli Kütüphanelerin Yüklenmesi**

Her veri bilimi projesinde olduğu gibi, ilk adımımız analiz için gerekli araçları (kütüphaneleri) çalışma ortamımıza dahil etmektir.

In [ ]:
# Bu hücre, eksik kütüphaneleri Kaggle ortamına yükleyecektir.
# Yükleme işlemi internet bağlantınıza bağlı olarak birkaç dakika sürebilir.
!pip install biopython
!pip install obonet
!pip install networkx
!pip install wordcloud

# Veri işleme ve analiz için temel kütüphaneler
import pandas as pd
import numpy as np

# Görselleştirme kütüphaneleri
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud

# Biyoinformatik ve özel dosya formatları için kütüphaneler
from Bio import SeqIO  # FASTA dosyalarını okumak için
import obonet         # OBO (Gene Ontology) dosyalarını okumak için
import networkx as nx # Graf (ağ) yapıları ile çalışmak için

# Yardımcı kütüphaneler
from tqdm.notebook import tqdm
import collections
import warnings

# Kod çıktılarının daha temiz görünmesi için uyarıları gizleyelim.
warnings.filterwarnings('ignore')
# Görselleştirmeler için varsayılan stili ayarlayalım.
sns.set_style('whitegrid')
# Grafiklerimizin varsayılan boyutunu ayarlayalım.
plt.rcParams['figure.figsize'] = (12, 6)

print("Tüm kütüphaneler başarıyla yüklendi ve import edildi. Analize başlayabilirsiniz.")

#### **1.2 Kavram Köşesi: Biyoinformatik Dünyasına Giriş**

Bu analizi daha iyi anlamak için bazı temel biyolojik ve hesaplamalı kavramları tanıyalım.

*   **Protein ve Amino Asitler:** Proteinler, DNA'da kodlanan genetik bilginin işlevsel ürünleridir. Canlılığın devamı için gerekli olan reaksiyonları katalizler, hücrelere yapısal destek sağlarlar ve sinyal iletiminde rol alırlar. Proteinler, **amino asit** adı verilen 20 farklı kimyasal yapı taşının belirli bir sırada bir araya gelmesiyle oluşan polimerlerdir. Bu sıralamaya **protein sekansı** denir. Bu sekans, proteinin üç boyutlu katlanmasını ve dolayısıyla biyolojik fonksiyonunu belirleyen en temel bilgidir. Yarışmadaki ana girdimiz de bu sekans bilgisidir.

*   **Gene Ontology (GO - Gen Ontolojisi):** Binlerce farklı protein ve fonksiyonu üzerine yapılan araştırmaları standartlaştırmak ve bilgisayarlar tarafından anlaşılabilir kılmak için geliştirilmiş bir sistemdir. GO, fonksiyonları üç ana alanda hiyerarşik olarak düzenler:
    *   **Moleküler Fonksiyon (MFO):** Bir proteinin moleküler seviyedeki aktivitesi. Örneğin, "ATP bağlama" (ATP binding) veya "kinaz aktivitesi" (kinase activity).
    *   **Biyolojik Süreç (BPO):** Bir proteinin dahil olduğu daha büyük biyolojik olaylar dizisi. Örneğin, "hücre döngüsü" (cell cycle) veya "glikoliz" (glycolysis).
    *   **Hücresel Bileşen (CCO):** Bir proteinin hücre içinde bulunduğu konum. Örneğin, "çekirdek" (nucleus) veya "mitokondri" (mitochondrion).
    Bu hiyerarşik yapı, fonksiyonlar arasında "bir türüdür" (is_a) veya "bir parçasıdır" (part_of) gibi ilişkiler kurar. Modelimizin tahmin etmesi gereken çıktılar bu GO terimleridir.

*   **Taksonomi:** Canlıların evrimsel ilişkilerine göre sınıflandırılması bilimidir. Her canlı türünün benzersiz bir **taksonomi ID**'si vardır. Örneğin, *Homo sapiens* (insan) için bu ID `9606`'dır. Protein fonksiyonları genellikle türler arasında korunur. Örneğin, insandaki bir metabolik enzimin benzeri genellikle farede veya mayada da bulunur. Bu evrimsel bilgi, fonksiyon tahmininde güçlü bir ipucu olabilir.

---

### **2. Veri Setini Tanıyalım**

Analize başlamadan önce, yarışma tarafından sağlanan tüm dosyaları ve yollarını tanımlayalım.

In [ ]:
# Veri dosyalarının bulunduğu ana dizin
data_path = '/kaggle/input/cafa-6-protein-function-prediction/'

# Dosya yolları
train_seq_path = f'{data_path}Train/train_sequences.fasta'
train_terms_path = f'{data_path}Train/train_terms.tsv'
train_tax_path = f'{data_path}Train/train_taxonomy.tsv'
test_seq_path = f'{data_path}Test/testsuperset.fasta'
go_obo_path = f'{data_path}Train/go-basic.obo'
ia_path = f'{data_path}IA.tsv'

| Dosya Adı | Açıklama |
| :--- | :--- |
| `train_sequences.fasta` | Eğitim setindeki proteinlerin amino asit dizilimleri. Modelimizin **girdi (input)** verisi. |
| `train_terms.tsv` | Eğitim setindeki proteinlerin atandığı GO terimleri. Modelimizin öğrenmesi gereken **hedef (target)** etiketleri. |
| `train_taxonomy.tsv` | Eğitim setindeki proteinlerin ait olduğu türlerin taksonomik ID'leri. Modele ek bilgi olarak sunulabilir. |
| `go-basic.obo` | Gene Ontology'nin hiyerarşik yapısını tanımlayan graf dosyası. Etiketler arasındaki ilişkileri içerir. |
| `testsuperset.fasta` | Fonksiyonunu tahmin etmemiz istenen test proteinlerinin dizilimleri. |
| `IA.tsv` | Her bir GO teriminin **Bilgi Birikimi (IA)** ağırlığı. Yarışma değerlendirme metriğinde kullanılır ve nadir fonksiyonların doğru tahminini ödüllendirir. |

---

### **3. Protein Sekansları Analizi (`train_sequences.fasta`)**

Modelimizin temel girdisi olan protein sekanslarının özelliklerini inceleyerek başlayalım.

In [ ]:
# FASTA dosyasını okumak için boş listeler oluşturalım.
protein_ids = []
seq_lengths = []
# Tüm amino asitleri saymak için bir Counter nesnesi oluşturalım.
amino_acid_counts = collections.Counter()

# SeqIO.parse ile FASTA dosyasını satır satır okuyalım.
# tqdm, döngünün ilerlemesini gösteren bir ilerleme çubuğu ekler.
for record in tqdm(SeqIO.parse(train_seq_path, "fasta"), desc="Eğitim Sekansları Okunuyor"):
    # Her bir proteinin ID'sini (örn: P9WHI7) başlık satırından alalım.
    protein_ids.append(record.id.split('|')[1])
    # Protein sekansının uzunluğunu (amino asit sayısı) listeye ekleyelim.
    seq_lengths.append(len(record.seq))
    # Sekanstaki her bir amino asidi sayalım.
    amino_acid_counts.update(record.seq)

# Protein ID'leri ve uzunluklarından bir pandas DataFrame'i oluşturalım.
train_seq_df = pd.DataFrame({
    'ProteinID': protein_ids,
    'Length': seq_lengths
})

# Temel istatistikleri yazdıralım.
print(f"Toplam eğitim proteini sayısı: {len(train_seq_df)}")
print("\nSekans Uzunluk İstatistikleri:")
# describe() fonksiyonu, bir sütunun temel istatistiksel özetini verir.
print(train_seq_df['Length'].describe())

# Sekans uzunluklarının dağılımını görselleştirelim.
# İki grafik (histogram ve kutu grafiği) oluşturmak için subplots kullanalım.
fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)
# Histogram, verinin belirli aralıklara ne sıklıkla düştüğünü gösterir.
sns.histplot(data=train_seq_df, x='Length', bins=100, ax=axes[0])
axes[0].set_title('Eğitim Seti Protein Sekans Uzunluk Dağılımı (Histogram)', fontsize=16)
axes[0].set_xlabel('')
axes[0].set_ylabel('Protein Sayısı')
# Dağılım çok geniş bir aralıkta olduğu için logaritmik ölçek kullanalım.
axes[0].set_xscale('log')

# Kutu grafiği, verinin çeyrekliklerini, medyanını ve aykırı değerlerini gösterir.
sns.boxplot(data=train_seq_df, x='Length', ax=axes[1])
axes[1].set_title('Eğitim Seti Protein Sekans Uzunluk Dağılımı (Boxplot)', fontsize=16)
axes[1].set_xlabel('Sekans Uzunluğu (Log Ölçek)', fontsize=12)
axes[1].set_xscale('log')

plt.tight_layout()
plt.show()

**Analiz ve Bulgular:**
*   Veri setinde **82,404** adet eğitim proteini bulunmaktadır.
*   Proteinlerin ortalama uzunluğu yaklaşık **526** amino asit. Ancak, standart sapmanın (521) çok yüksek olması ve maksimum uzunluğun **35,213**'e (Titin gibi devasa bir kas proteini olabilir) ulaşması, uzunluk dağılımının çok geniş bir aralıkta olduğunu gösteriyor. Proteinlerin farklı fonksiyonları yerine getirmek için farklı boyutlarda ve karmaşıklıkta olması gerektiğinden bu durum biyolojik olarak beklenir.
*   Grafikler, dağılımın **sağa çarpık (uzun kuyruklu)** olduğunu doğruluyor. Bu, proteinlerin büyük bir kısmının görece kısa olduğunu, ancak çok az sayıda aşırı uzun proteinin bulunduğunu gösterir.
*   **Modelleme İçin Anlamı:** Makine öğrenmesi modelleri, özellikle Transformer tabanlı olanlar, genellikle sabit uzunlukta girdilerle çalışır. Bu aşırı uzun sekansları nasıl işleyeceğimiz (örneğin, sekansı kesmek, parçalara ayırmak veya bu uzunluk değişkenliğini yönetebilen RNN/CNN gibi modeller kullanmak) önemli bir modelleme kararıdır.

#### **Amino Asit Frekansları**

Şimdi de tüm eğitim setindeki 20 temel amino asidin genel dağılımına bakalım.

In [ ]:
# Amino asit sayılarını bir DataFrame'e dönüştürelim ve sıklığa göre sıralayalım.
aa_freq_df = pd.DataFrame(amino_acid_counts.items(), columns=['AminoAcid', 'Count']).sort_values('Count', ascending=False)

# Sıklığı bir çubuk grafiği ile görselleştirelim.
plt.figure(figsize=(16, 8))
sns.barplot(data=aa_freq_df, x='AminoAcid', y='Count', palette='viridis')
plt.title('Eğitim Setindeki Toplam Amino Asit Sıklığı', fontsize=16)
plt.xlabel('Amino Asit (Tek Harf Kodu)', fontsize=12)
plt.ylabel('Toplam Sayı', fontsize=12)
plt.show()

**Analiz ve Bulgular:**
*   En sık rastlanan amino asitler Lösin (L), Serin (S) ve Alanin (A) gibi küçük veya hidrofobik amino asitlerdir.
*   En nadir amino asitler ise Triptofan (W) ve Sistein (C) gibi daha karmaşık veya kimyasal olarak reaktif olanlardır.
*   Bu dağılım, bilinen tüm proteomlardaki genel amino asit dağılımı ile tutarlıdır. Bu, veri setimizin biyolojik olarak geçerli ve yüksek kalitede olduğuna dair bir başka kanıttır.

---

### **4. Fonksiyonel Etiketler Analizi (`train_terms.tsv`)**

Bu bölümde, modelimizin neyi tahmin etmesi gerektiğini, yani proteinlerin fonksiyonel "etiketlerini" inceleyeceğiz. Bu analiz, yarışma probleminin yapısını anlamak için kritik öneme sahiptir.

In [ ]:
# Eğitim etiketleri dosyasını (train_terms.tsv) bir pandas DataFrame'i olarak okuyalım.
train_terms_df = pd.read_csv(train_terms_path, sep='\t')

# Veri setindeki temel sayıları yazdıralım.
print(f"Toplam etiket ataması sayısı: {len(train_terms_df)}")
print(f"Etiketlenmiş benzersiz protein sayısı: {train_terms_df['EntryID'].nunique()}")
print(f"Benzersiz GO terimi (etiket) sayısı: {train_terms_df['term'].nunique()}")
print("\nVeri Örneği:")
print(train_terms_df.head())

# Üç ana GO ontolojisine (P, C, F) göre etiketlerin dağılımını sayalım.
# Daha anlaşılır olması için kısaltmaları uzun isimleriyle eşleştirelim.
ontology_mapping = {'P': 'Biyolojik Süreç (BPO)', 'C': 'Hücresel Bileşen (CCO)', 'F': 'Moleküler Fonksiyon (MFO)'}
train_terms_df['aspect_long'] = train_terms_df['aspect'].map(ontology_mapping)
ontology_counts = train_terms_df['aspect_long'].value_counts()

# Pasta grafiği ile ontoloji dağılımını görselleştirelim.
fig = px.pie(ontology_counts, values=ontology_counts.values, names=ontology_counts.index,
             title='GO Ontoloji Sınıflarına Göre Etiket Dağılımı',
             color_discrete_sequence=px.colors.sequential.RdBu)
# Grafik dilimlerinin içine yüzde ve etiket adını yazdıralım.
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

**Analiz ve Bulgular:**
*   Veri setinde toplam **537,027** fonksiyon ataması (etiket) bulunmaktadır. Bu atamalar **82,404** benzersiz proteine yapılmıştır.
*   Bu etiketler **26,125** benzersiz GO terimine aittir. Bu, tahmin edilecek etiket sayısının çok yüksek olduğunu ve problemin yüksek boyutlu bir sınıflandırma görevi olduğunu gösterir.
*   Etiketlerin çoğunluğu (%46.7) **Biyolojik Süreç (BPO)** ontolojisine aittir. Bu genellikle en karmaşık ve en çok terim içeren daldır. Bunu **Hücresel Bileşen (CCO)** (%29.4) ve **Moleküler Fonksiyon (MFO)** (%23.9) takip eder. Bu dağılım, modelimizin her bir ontoloji için farklı zorluklarla karşılaşabileceğini ve belki de her biri için farklı stratejiler gerektirebileceğini ima eder.

#### **Etiket Dağılımları: Çok Etiketlilik ve Dengesizlik**

**Kavram Köşesi:**
*   **Çok Etiketli Sınıflandırma (Multi-label Classification):** Geleneksel sınıflandırma problemlerinde her örneğin tek bir etiketi vardır (kedi, köpek gibi). Ancak biyolojide bir protein aynı anda birden fazla göreve sahip olabilir. Örneğin, hem "çekirdekte" (CCO) bulunabilir hem de "DNA'ya bağlanma" (MFO) fonksiyonunu yerine getirebilir. Her bir örneğe birden fazla etiket atanabilen bu tür problemlere **çok etiketli sınıflandırma** denir.
*   **Etiket Dengesizliği (Label Imbalance):** Veri setinde bazı etiketlerin çok sık, bazılarının ise çok nadir görülmesi durumudur. Bu, makine öğrenmesi modelleri için büyük bir zorluktur çünkü model, sık görülen etiketleri öğrenmeye odaklanıp nadir olanları göz ardı etme eğiliminde olur.

Şimdi bu kavramların veri setimizde ne kadar belirgin olduğunu inceleyelim.

In [ ]:
# Her bir protein için kaç tane etiket olduğunu sayalım.
labels_per_protein = train_terms_df.groupby('EntryID')['term'].count()

# Protein başına etiket sayısının dağılımını histogram ile gösterelim.
plt.figure(figsize=(16, 6))
# Dikey eksende logaritmik ölçek kullanarak nadir durumları (çok fazla etiketi olan proteinler) daha iyi görelim.
sns.histplot(labels_per_protein, bins=100, log_scale=(False, True))
plt.title('Protein Başına Atanan GO Terimi Sayısı Dağılımı', fontsize=16)
plt.xlabel('Bir Proteine Atanan Etiket Sayısı', fontsize=12)
plt.ylabel('Protein Sayısı (Log Ölçek)', fontsize=12)
plt.show()
print("\nProtein Başına Etiket Sayısı İstatistikleri:")
print(labels_per_protein.describe())

# Her bir etiketin kaç farklı proteine atandığını sayalım (etiket popülerliği).
proteins_per_label = train_terms_df.groupby('term')['EntryID'].count().sort_values(ascending=False)

# Etiket popülerliğinin dağılımını histogram ile gösterelim.
plt.figure(figsize=(16, 6))
# Hem yatay hem dikey eksende logaritmik ölçek kullanalım, çünkü dağılım çok geniş bir aralıkta.
sns.histplot(proteins_per_label, bins=100, log_scale=(True, True))
plt.title('GO Terimi Başına Protein Sayısı Dağılımı (Etiket Popülerliği)', fontsize=16)
plt.xlabel('Bir GO Teriminin Atandığı Protein Sayısı (Log Ölçek)', fontsize=12)
plt.ylabel('GO Terimi Sayısı (Log Ölçek)', fontsize=12)
plt.show()
print("\nEn Popüler 10 GO Terimi:")
print(proteins_per_label.head(10))

**Analiz ve Bulgular:**
*   **Protein Başına Etiket Sayısı:** Bir proteine ortalama **6.5** etiket atanmış, ancak bazı proteinler **233**'e kadar etikete sahip. Bu durum, problemin kesinlikle **çok etiketli (multi-label)** bir sınıflandırma problemi olduğunu güçlü bir şekilde doğrular. Modelimiz, her protein için birden fazla "evet" cevabı verebilmelidir.
*   **Etiket Popülerliği:** Bu grafik, yarışmanın en büyük zorluklarından birini gözler önüne seriyor: **aşırı etiket dengesizliği**. "protein binding" (GO:0005515) gibi çok genel ve GO hiyerarşisinin tepesine yakın terimler on binlerce proteine atanmışken, GO terimlerinin ezici çoğunluğu sadece birkaç proteine atanmıştır. Bu, modelin nadir fonksiyonları öğrenmesini zorlaştıracak ve özel kayıp fonksiyonları (weighted loss) veya örnekleme stratejileri (sampling strategies) gibi çözümler gerektirecektir.

---

### **5. Taksonomi Analizi (`train_taxonomy.tsv`)**

**Kavram Köşesi:**
*   **Model Organizmalar:** Biyolojik araştırmalarda, insan biyolojisini ve hastalıklarını anlamak için kullanılan, genetik olarak iyi karakterize edilmiş ve laboratuvarda çalışması kolay olan canlılardır (örn: fare, maya, meyve sineği). Proteomik veri tabanları genellikle bu organizmalardan gelen verilerle zengindir.
*   **Homoloji ve Fonksiyon:** Evrimsel olarak ortak bir atadan gelen genlere (ve proteinlere) **homolog** denir. Homolog proteinler genellikle benzer fonksiyonlara sahiptir. Bu nedenle, bir proteinin hangi türden geldiği bilgisi, onun fonksiyonunu tahmin etmek için değerli bir ipucu olabilir.

In [ ]:
# Taksonomi dosyasını okurken başlık satırı olmadığını ve sütun isimlerini manuel olarak belirtelim.
train_tax_df = pd.read_csv(train_tax_path, sep='\t', names=['ProteinID', 'taxonomyID'])

print(f"Benzersiz takson ID sayısı: {train_tax_df['taxonomyID'].nunique()}")

# En sık görülen 20 türü bulalım.
top_20_species = train_tax_df['taxonomyID'].value_counts().nlargest(20)

# Çubuk grafiği ile görselleştirelim.
plt.figure(figsize=(16, 8))
# Takson ID'leri sayısal olduğu için grafikte string olarak gösterelim ki kategorik algılansın.
sns.barplot(x=top_20_species.index.astype(str), y=top_20_species.values, palette='magma')
plt.title('Eğitim Setindeki En Yaygın 20 Tür (Takson ID)', fontsize=16)
plt.xlabel('Taksonomi ID', fontsize=12)
plt.ylabel('Protein Sayısı', fontsize=12)
plt.xticks(rotation=45)
plt.show()

print("\nEn yaygın türler ve bilimsel adları:")
print("9606: Homo sapiens (İnsan)")
print("10090: Mus musculus (Fare)")
print("10116: Rattus norvegicus (Sıçan)")
print("4932: Saccharomyces cerevisiae (Maya)")
print("7227: Drosophila melanogaster (Meyve Sineği)")

**Analiz ve Bulgular:**
*   Veri setindeki proteinler **1,381** farklı canlı türüne aittir, bu da veri setinin taksonomik olarak çeşitli olduğunu gösterir.
*   Ancak, grafikten de görüleceği gibi, veri seti **model organizmalar** üzerinde yoğunlaşmıştır. En baskın türler insan, fare, sıçan, maya ve meyve sineğidir.
*   **Modelleme İçin Anlamı:** Modelimiz, bu sık görülen türler için daha iyi performans gösterebilir. Ancak, daha az temsil edilen türler için genelleme yapmakta zorlanabilir. Taksonomi bilgisini modele ek bir özellik olarak sunmak, bu genelleme yeteneğini artırabilir.

---

### **6. Gene Ontology (GO) Hiyerarşisi (`go-basic.obo`)**

**Kavram Köşesi:**
*   **Yönlendirilmiş Döngüsüz Graf (DAG - Directed Acyclic Graph):** GO hiyerarşisi bir ağaç gibi değildir, çünkü bir terimin birden fazla "ebeveyni" olabilir. Bu tür bir yapıya DAG denir. Bu yapı, fonksiyonlar arasında karmaşık ve çoklu ilişkiler kurulmasına olanak tanır.
*   **Doğruluk Kuralı (True Path Rule):** GO'nun temel kurallarından biridir. Eğer bir protein, bir GO terimi ile etiketlenmişse, o terimin DAG üzerindeki tüm üst (ata) terimleri ile de otomatik olarak etiketlenmiş sayılır. Örneğin, "pozitif transkripsiyon regülasyonu" fonksiyonuna sahip bir protein, aynı zamanda "transkripsiyon regülasyonu" ve "biyolojik regülasyon" gibi daha genel fonksiyonlara da sahiptir.

In [ ]:
# OBO dosyasını bir networkx grafiği olarak yükleyelim. Bu, terimler arasındaki ilişkileri analiz etmemizi sağlar.
go_graph = obonet.read_obo(go_obo_path)

print(f"GO Grafiğindeki Düğüm (Terim) Sayısı: {go_graph.number_of_nodes()}")
print(f"GO Grafiğindeki Kenar (İlişki) Sayısı: {go_graph.number_of_edges()}")

# Örnek bir terimin ("protein binding") graf üzerindeki bilgilerini gösterelim.
sample_term = 'GO:0005515'
if go_graph.has_node(sample_term):
    print(f"\nÖrnek Terim Bilgisi ({sample_term}):")
    # 'is_a' ilişkisi, bu terimin hangi daha genel terimin bir alt türü olduğunu gösterir.
    print(go_graph.nodes[sample_term])

# Üç ana ontolojinin kök düğümlerini tanımlayalım. Bu düğümler hiyerarşinin en tepesindedir.
subontology_roots = {
    'BPO': 'GO:0008150', # biological_process
    'CCO': 'GO:0005575', # cellular_component
    'MFO': 'GO:0003674'  # molecular_function
}

---

#### **Terim Derinliği ve Bilgi Birikimi (IA) Analizi**

**Kavram Köşesi:**
*   **Terim Derinliği (Term Depth):** Bir GO teriminin, hiyerarşinin en tepesindeki kök terime olan en kısa yol uzunluğudur. Derinlik arttıkça, terim daha spesifik ve özelleşmiş bir fonksiyonu tanımlar. Örneğin, "bağlanma" (binding) terimi sığ bir terimken, "insülin reseptörüne bağlanma" (insulin receptor binding) çok daha derin ve spesifik bir terimdir.
*   **Bilgi Birikimi (IA - Information Accretion):** Bu, yarışmanın değerlendirme metriğinde kullanılan özel bir ağırlıklandırma değeridir. Bir terimin IA değeri, o terimin bir veri tabanında ne kadar nadir görüldüğüne bağlıdır. Nadir görülen (dolayısıyla daha spesifik ve tahmin etmesi zor olan) terimlerin IA değeri daha yüksektir. Yarışmada başarılı olmak, sadece popüler terimleri değil, IA değeri yüksek olan nadir terimleri de doğru tahmin etmeyi gerektirir.

Şimdi bu iki metrik arasındaki ilişkiyi inceleyelim. Beklentimiz, derinlik arttıkça IA değerinin de artması yönündedir.

In [ ]:
# IA dosyasını yükleyip bir sözlük yapısına dönüştürelim. Bu, terim ID'si ile IA değerine hızlıca erişmemizi sağlar.
ia_df = pd.read_csv(ia_path, sep='\t', names=['term', 'IA'])
ia_map = ia_df.set_index('term')['IA'].to_dict()

# Her terimin derinliğini hesaplamak için boş bir sözlük oluşturalım.
depths = {}
# Her bir ontoloji kökü (BPO, CCO, MFO) için döngü başlatalım.
for aspect, root in subontology_roots.items():
    if go_graph.has_node(root):
        # Kök düğümden erişilebilen tüm alt düğümleri (torunları) bulalım.
        descendants = nx.ancestors(go_graph, root)
        descendants.add(root)
        # Sadece bu ontolojiye ait alt grafiği kullanarak derinliği hesaplayalım.
        subgraph = go_graph.subgraph(descendants)
        for node in subgraph.nodes():
            try:
                # networkx kütüphanesi ile köke olan en kısa yol uzunluğunu (derinliği) bulalım.
                depths[node] = nx.shortest_path_length(subgraph, source=root, target=node)
            except nx.NetworkXNoPath:
                # Bazı düğümler (artık kullanılmayan eski terimler gibi) köke bağlı olmayabilir, onları atlayalım.
                pass

# Derinlik ve IA değerlerini bir DataFrame'de birleştirelim.
depth_ia_data = []
for term, depth in depths.items():
    if term in ia_map: # Sadece IA değeri olan terimleri dahil edelim.
        depth_ia_data.append({'term': term, 'depth': depth, 'IA': ia_map[term]})
depth_ia_df = pd.DataFrame(depth_ia_data)

# Saçılım grafiği (scatterplot) ile Derinlik ve IA arasındaki ilişkiyi görselleştirelim.
plt.figure(figsize=(12, 8))
sns.scatterplot(data=depth_ia_df, x='depth', y='IA', alpha=0.5)
# İlişkinin genel eğilimini görmek için bir regresyon çizgisi ekleyelim.
sns.regplot(data=depth_ia_df, x='depth', y='IA', scatter=False, color='red', line_kws={'linewidth': 2})
plt.title('GO Terim Derinliği ve Bilgi Birikimi (IA) İlişkisi', fontsize=16)
plt.xlabel('Derinlik (Kökten Uzaklık)', fontsize=12)
plt.ylabel('Bilgi Birikimi (IA)', fontsize=12)
plt.show()

**Analiz ve Bulgular:**
*   Grafik, beklentimizle uyumlu olarak, bir terimin derinliği arttıkça (yani daha spesifik hale geldikçe), IA değerinin de genel olarak arttığını gösteriyor. Bu, spesifik fonksiyonların daha nadir ve dolayısıyla daha "bilgilendirici" olduğu anlamına gelir.
*   Ancak, bu ilişki mükemmel bir doğru değildir. Aynı derinlikte çok farklı IA değerlerine sahip birçok terim vardır. Bu durum, bazı biyolojik alanların diğerlerinden daha yoğun bir şekilde araştırıldığını ve etiketlendiğini gösterir. Örneğin, belirli bir sinyal yolundaki tüm proteinler benzer derinlikte olabilir, ancak bazıları çok daha nadir olduğu için daha yüksek IA değerine sahip olabilir.
*   **Modelleme İçin Anlamı:** Modelimizin başarısı, IA değeri yüksek (nadir ve değerli) terimleri doğru tahmin etme yeteneğine bağlıdır. Bu nedenle, etiket dengesizliği sorununu ele alan ve nadir etiketlere daha fazla önem veren modelleme stratejileri (örneğin, ağırlıklı kayıp fonksiyonları) kullanmak kritik olacaktır.

---

### **7. Bütünsel Bakış: Eğitim ve Test Setlerinin Karşılaştırılması**

**Kavram Köşesi:**
*   **Dağılım Kayması (Domain Shift / Distribution Shift):** Bir makine öğrenmesi modelinin eğitildiği veri (eğitim seti) ile test edildiği verinin (test seti) istatistiksel özelliklerinin birbirinden farklı olması durumudur. Bu, modelin gerçek dünya performansının beklenenden düşük olmasına neden olan yaygın bir sorundur. Örneğin, sadece kısa proteinlerle eğitilmiş bir model, test setinde çok uzun proteinlerle karşılaştığında başarısız olabilir. Bu nedenle, eğitim ve test setlerinin ne kadar benzer olduğunu kontrol etmek önemlidir.

In [ ]:
# Test setindeki sekansları okuyalım ve uzunluklarını bir listeye kaydedelim.
test_protein_ids = []
test_seq_lengths = []

# tqdm ile ilerleme çubuğu ekleyerek test sekanslarını okuyalım.
for record in tqdm(SeqIO.parse(test_seq_path, "fasta"), desc="Test Sekansları Okunuyor"):
    test_protein_ids.append(record.id)
    test_seq_lengths.append(len(record.seq))

# Test sekanslarından bir DataFrame oluşturalım.
test_seq_df = pd.DataFrame({
    'ProteinID': test_protein_ids,
    'Length': test_seq_lengths
})

print(f"Toplam test proteini sayısı: {len(test_seq_df)}")

# Eğitim ve test setlerindeki protein uzunluk dağılımlarını karşılaştıralım.
# Kernel Yoğunluk Tahmini (KDE) grafiği, verinin sürekli olasılık yoğunluğunu gösterir ve dağılımları karşılaştırmak için idealdir.
plt.figure(figsize=(16, 8))
sns.kdeplot(train_seq_df['Length'], label='Eğitim Seti', fill=True, color='blue')
sns.kdeplot(test_seq_df['Length'], label='Test Seti', fill=True, color='red')
plt.title('Eğitim ve Test Seti Protein Uzunluk Dağılımlarının Karşılaştırılması', fontsize=16)
plt.xlabel('Sekans Uzunluğu (Log Ölçek)', fontsize=12)
plt.ylabel('Yoğunluk', fontsize=12)
# Uzun kuyruklu dağılımı daha iyi görmek için x ekseninde logaritmik ölçek kullanalım.
plt.xscale('log')
plt.legend()
plt.show()

**Analiz ve Bulgular:**
*   Eğitim ve test setlerindeki protein uzunluk dağılımları görsel olarak birbirine **çok benziyor**.
*   Bu, yarışma organizatörlerinin veri setlerini benzer özelliklere sahip olacak şekilde böldüğünü gösteren **çok olumlu bir bulgudur**. Bu durum, modelimizin eğitim verisinde öğrendiği sekans uzunluğuna bağlı özelliklerin, test verisi için de geçerli olacağı ve ciddi bir **dağılım kayması** sorunu yaşamayacağımız anlamına gelir. Modelimizin genelleme yeteneği konusunda bu bize güven verir.

---

### **8. İleri Düzey Görselleştirmeler: İnteraktif GO İlişki Ağı**

Şimdi, en popüler 100 GO terimini ve aralarındaki hiyerarşik ilişkileri gösteren interaktif bir ağ grafiği oluşturalım. Bu görselleştirme, ontolojinin karmaşık yapısını ve fonksiyonların birbirleriyle nasıl kümelendiğini sezgisel olarak anlamamızı sağlayacak.

In [ ]:
# Daha önce hesapladığımız etiket popülerliği listesinden en popüler 100 terimi alalım.
top_100_terms = proteins_per_label.head(100).index.tolist()

# Bu terimleri ve onların tüm atalarını (üst terimlerini) içeren bir alt graf oluşturalım.
# Ataları da dahil etmek, terimler arasındaki hiyerarşik yolları görmemizi sağlar.
subgraph_nodes = set(top_100_terms)
for term in top_100_terms:
    if go_graph.has_node(term):
        ancestors = nx.ancestors(go_graph, term)
        subgraph_nodes.update(ancestors)
subgraph = go_graph.subgraph(list(subgraph_nodes))

# Görselleştirmeyi daha temiz hale getirmek için sadece en popüler 100 terim arasındaki kenarları tutan yeni bir graf oluşturalım.
display_graph = nx.DiGraph()
display_graph.add_nodes_from(top_100_terms)
for u, v in subgraph.edges():
    if u in top_100_terms and v in top_100_terms:
        display_graph.add_edge(u, v)

# Plotly kütüphanesi ile interaktif bir çizim yapalım.
if not nx.is_empty(display_graph):
    # Düğümlerin konumunu belirlemek için bir "yay düzeni" (spring layout) algoritması kullanalım.
    # Bu algoritma, düğümleri birbirine bağlı olanları yakın, olmayanları uzak olacak şekilde yerleştirir.
    pos = nx.spring_layout(display_graph, k=0.5, iterations=50)

    # Kenarları (bağlantıları) çizmek için koordinatları hazırlayalım.
    edge_x, edge_y = [], []
    for edge in display_graph.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None]) # 'None' eklemek, plotly'nin çizgileri ayırmasını sağlar.
        edge_y.extend([y0, y1, None])
    edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='#888'), hoverinfo='none', mode='lines')

    # Düğümleri (terimleri) çizmek için koordinatları ve diğer özellikleri hazırlayalım.
    node_x, node_y, node_text, node_color, node_size = [], [], [], [], []
    for node in display_graph.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        
        # Fare ile üzerine gelince gösterilecek bilgileri (isim, ID, protein sayısı, ontoloji) hazırlayalım.
        term_name = go_graph.nodes.get(node, {}).get('name', node)
        protein_count = proteins_per_label.get(node, 0)
        term_aspect_code = train_terms_df[train_terms_df['term'] == node]['aspect'].iloc[0] if not train_terms_df[train_terms_df['term'] == node].empty else 'N/A'
        term_aspect_long = ontology_mapping.get(term_aspect_code, 'N/A')
        
        node_text.append(f"{term_name}<br>ID: {node}<br>Protein Sayısı: {protein_count}<br>Ontoloji: {term_aspect_long}")
        # Düğümün boyutu, popülerliğine (protein sayısı) logaritmik olarak bağlı olsun.
        node_size.append(10 + np.log1p(protein_count))
        
        # Düğümün rengi, ait olduğu ontolojiye (BPO, CCO, MFO) bağlı olsun.
        if term_aspect_code == 'P':
            node_color.append('rgba(255, 127, 14, 0.8)') # Turuncu
        elif term_aspect_code == 'F':
            node_color.append('rgba(44, 160, 44, 0.8)') # Yeşil
        elif term_aspect_code == 'C':
            node_color.append('rgba(31, 119, 180, 0.8)') # Mavi
        else:
            node_color.append('rgba(127, 127, 127, 0.8)') # Gri
    node_trace = go.Scatter(x=node_x, y=node_y, mode='markers', hoverinfo='text', marker=dict(showscale=False, color=node_color, size=node_size, line_width=2))
    node_trace.text = node_text

    # Grafiği oluşturalım ve gösterelim.
    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title='<br>En Popüler 100 GO Terimi Arasındaki İlişki Ağı (İnteraktif)', titlefont_size=16, showlegend=False,
                    hovermode='closest', margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
    fig.show()
else:
    print("Görselleştirilecek ağ boş.")

**Analiz ve Bulgular:**
*   Bu interaktif graf, farklı ontolojilerin (renklerle gösterilen) nasıl birbirleriyle ilişkili olduğunu ve genel terimlerin (daha büyük düğümler) daha spesifik terimlere nasıl bağlandığını (ok yönüyle) gösterir.
*   Grafiğin merkezinde, "nucleus" (çekirdek), "cytoplasm" (sitoplazma), "protein binding" (protein bağlanması) gibi çok genel ve çok sayıda proteine atanmış olan "merkez (hub)" terimlerin kümelendiği görülmektedir.
*   Fare ile üzerine gelerek her bir terimin adını, atandığı protein sayısını ve ait olduğu ontolojiyi görebilirsiniz. Bu görselleştirme, fonksiyonel annotasyonların karmaşık ve hiyerarşik doğasını keşfetmek için güçlü bir araçtır.

---

### **9. Sonuç ve Modelleme Stratejileri**

Bu öğretici ve kapsamlı Keşifsel Veri Analizi'nden elde ettiğimiz temel bulgular ve modelleme için çıkarımlar şunlardır:

1.  **Problem Yapısı:** Problemin, her proteine birden fazla fonksiyon atanabilen **çok etiketli (multi-label) bir sınıflandırma** problemi olduğunu doğruladık.
2.  **Girdi Verisi:** Girdi olarak değişken uzunlukta protein sekansları veriliyor. Bu, özellikle Transformer tabanlı modeller için sekansların nasıl işleneceği (kesme, parçalama) konusunda bir strateji gerektiriyor.
3.  **Etiket Dengesizliği:** Veri setindeki en büyük zorluk, etiket dağılımının aşırı derecede **uzun kuyruklu** olmasıdır. Nadir fonksiyonları tahmin etmek, model performansını belirleyen ana faktör olacaktır.
    *   **Strateji:** Etiket sıklığına göre ağırlıklandırılmış kayıp fonksiyonları (weighted loss functions), azınlık sınıfları için over-sampling (örnek sayısını artırma) veya daha karmaşık hiyerarşik sınıflandırma yaklaşımları denenmelidir.
4.  **Hiyerarşik Etiket Yapısı:** GO terimleri bir **Yönlendirilmiş Döngüsüz Graf (DAG)** yapısındadır. Bir alt terimin tahmini, üst terimlerinin de tahmin edilmesi anlamına gelir ("doğruluk kuralı").
    *   **Strateji:** Modelin tahminlerinden sonra, bu hiyerarşik bilgiyi kullanarak tahminleri tutarlı hale getirmek (örneğin, bir alt terim tahmin edildiyse tüm üst terimlerini de tahmin etmek) zorunludur. Buna **tahmin yayılımı (prediction propagation)** denir.
5.  **Değerlendirme Metriği:** Yarışma, nadir ve bilgilendirici terimleri ödüllendiren **IA (Bilgi Birikimi)** ile ağırlıklandırılmış bir F-max skoru kullanmaktadır.
    *   **Strateji:** Model, sadece sık görülen genel terimleri değil, IA değeri yüksek terimleri de öğrenmeye odaklanmalıdır. Bu, etiket dengesizliği ile başa çıkma stratejileriyle doğrudan ilişkilidir.
6.  **Veri Dağılımı:** Eğitim ve test setleri arasında sekans uzunluğu açısından büyük bir **dağılım kayması (domain shift)** olmaması, modelin test setinde de iyi bir genelleme yapabileceği konusunda bize güven vermektedir.
7.  **Ontoloji Farklılıkları:** Üç ana ontoloji dalı (BPO, CCO, MFO) farklı sayıda etiket ve farklı karmaşıklık seviyeleri içermektedir.
    *   **Strateji:** Her bir ontoloji için ayrı modeller eğitmek veya tek bir modelin sonunda her ontoloji için ayrı bir "sınıflandırma başlığı" (classification head) kullanmak, her dalın kendine özgü yapısını daha iyi öğrenmeye yardımcı olabilir.

Bu EDA, CAFA 6 yarışmasında başarılı olmak ve protein fonksiyonu tahmini alanını anlamak için sağlam bir temel oluşturmaktadır. Sonraki adımlar, bu bulgular ışığında uygun bir model mimarisi seçmek (örneğin, ProtBERT gibi önceden eğitilmiş bir protein dil modeli), veriyi modele uygun hale getirmek ve sağlam bir doğrulama (validation) stratejisi oluşturmaktır.